# Customize state
## 1. Add keys to the state

In [ ]:
from typing import Annotated

from typing_extensions import TypedDict

from langgraph.graph.message import add_messages


class State(TypedDict):
    messages: Annotated[list, add_messages]
    name: str
    birthday: str

## 2. Update the state inside the tool

In [ ]:
from langchain_core.messages import ToolMessage
from langchain_core.tools import InjectedToolCallId, tool

from langgraph.types import Command, interrupt

@tool
# Note that because we are generating a ToolMessage for a state update, we
# generally require the ID of the corresponding tool call. We can use
# LangChain's InjectedToolCallId to signal that this argument should not
# be revealed to the model in the tool's schema.
def human_assistance(
    name: str, birthday: str, tool_call_id: Annotated[str, InjectedToolCallId]
) -> str:
    """Request assistance from a human."""
    human_response = interrupt(
        {
            "question": "Is this correct?",
            "name": name,
            "birthday": birthday,
        },
    )
    # If the information is correct, update the state as-is.
    if human_response.get("correct", "").lower().startswith("y"):
        verified_name = name
        verified_birthday = birthday
        response = "Correct"
    # Otherwise, receive information from the human reviewer.
    else:
        verified_name = human_response.get("name", name)
        verified_birthday = human_response.get("birthday", birthday)
        response = f"Made a correction: {human_response}"

    # This time we explicitly update the state with a ToolMessage inside
    # the tool.
    state_update = {
        "name": verified_name,
        "birthday": verified_birthday,
        "messages": [ToolMessage(response, tool_call_id=tool_call_id)],
    }
    # We return a Command object in the tool to update our state.
    return Command(update=state_update)

## 3. Prompt the chatbot

In [ ]:
user_input = (
    "Can you look up when LangGraph was released? "
    "When you have the answer, use the human_assistance tool for review."
)
config = {"configurable": {"thread_id": "1"}}

events = graph.stream(
    {"messages": [{"role": "user", "content": user_input}]},
    config,
    stream_mode="values",
)
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

## 4. Add human assistance

In [ ]:
human_command = Command(
    resume={
        "name": "LangGraph",
        "birthday": "Jan 17, 2024",
    },
)

events = graph.stream(human_command, config, stream_mode="values")
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

In [ ]:
snapshot = graph.get_state(config)

{k: v for k, v in snapshot.values.items() if k in ("name", "birthday")}

## 5. Manually update the state

In [ ]:
graph.update_state(config, {"name": "LangGraph (library)"})

## 6. View the new value

In [ ]:
snapshot = graph.get_state(config)

{k: v for k, v in snapshot.values.items() if k in ("name", "birthday")}

# 完整代码

In [1]:
from typing import Annotated

from langchain_tavily import TavilySearch
from langchain_core.messages import ToolMessage
from langchain_core.tools import InjectedToolCallId, tool
from typing_extensions import TypedDict

from langgraph.checkpoint.memory import InMemorySaver
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.types import Command, interrupt
from langchain_openai import ChatOpenAI

import os

os.environ["TAVILY_API_KEY"] = "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"

llm = ChatOpenAI(
        # model="Qwen/Qwen2.5-72B-Instruct",
        model="Qwen/Qwen3-8B",
        # 硅基流动
        api_key="sk-jvjyawqpodlkxlywatvemcdykkrbvthhjyjyapyvtnifwlbl",
        base_url="https://api.siliconflow.cn/v1/",
        # # modelscope 
        # base_url="https://api-inference.modelscope.cn/v1/",
        # api_key="ms-e2666046-2f3b-4c76-bcc0-e21f8ebf9ea1",
    )

class State(TypedDict):
    messages: Annotated[list, add_messages]
    name: str
    birthday: str

@tool
def human_assistance(
    name: str, birthday: str, tool_call_id: Annotated[str, InjectedToolCallId]
) -> str:
    """Request assistance from a human."""
    human_response = interrupt(
        {
            "question": "Is this correct?",
            "name": name,
            "birthday": birthday,
        },
    )
    if human_response.get("correct", "").lower().startswith("y"):
        verified_name = name
        verified_birthday = birthday
        response = "Correct"
    else:
        verified_name = human_response.get("name", name)
        verified_birthday = human_response.get("birthday", birthday)
        response = f"Made a correction: {human_response}"

    state_update = {
        "name": verified_name,
        "birthday": verified_birthday,
        "messages": [ToolMessage(response, tool_call_id=tool_call_id)],
    }
    return Command(update=state_update)


tool = TavilySearch(max_results=2)
tools = [tool, human_assistance]
llm_with_tools = llm.bind_tools(tools)

def chatbot(state: State):
    message = llm_with_tools.invoke(state["messages"])
    assert(len(message.tool_calls) <= 1)
    return {"messages": [message]}

graph_builder = StateGraph(State)
graph_builder.add_node("chatbot", chatbot)

tool_node = ToolNode(tools=tools)
graph_builder.add_node("tools", tool_node)

graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")

memory = InMemorySaver()
graph = graph_builder.compile(checkpointer=memory)

In [2]:
user_input = (
    "Can you look up when LangGraph was released? "
    "When you have the answer, use the human_assistance tool for review."
)
config = {"configurable": {"thread_id": "1"}}

events = graph.stream(
    {"messages": [{"role": "user", "content": user_input}]},
    config,
    stream_mode="values",
)
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

================================ Human Message =================================

Can you look up when LangGraph was released? When you have the answer, use the human_assistance tool for review.
================================== Ai Message ==================================
Tool Calls:
  tavily_search (0198e61441983ae72eac646c895171ea)
 Call ID: 0198e61441983ae72eac646c895171ea
  Args:
    query: LangGraph release date
    search_depth: advanced
    time_range: year
================================= Tool Message =================================
Name: tavily_search

{"query": "LangGraph release date", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://en.wikipedia.org/wiki/LangChain", "title": "LangChain", "content": "| Connery | No | Yes | Commercial | API actions | https://python.langchain.com/docs/integrations/tools/connery | | Dall-E Image Generator | No | Yes | Proprietary | Text-to-image generation | https://python.langchain.com/docs/integrat

In [3]:
human_command = Command(
    resume={
        "name": "LangGraph",
        "birthday": "Jan 17, 2024",
    },
)

events = graph.stream(human_command, config, stream_mode="values")
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

================================== Ai Message ==================================
Tool Calls:
  human_assistance (0198e615fbfaa91e8de1175253bf8644)
 Call ID: 0198e615fbfaa91e8de1175253bf8644
  Args:
    name: LangGraph
    birthday: release date
================================= Tool Message =================================
Name: human_assistance

Made a correction: {'name': 'LangGraph', 'birthday': 'Jan 17, 2024'}
================================== Ai Message ==================================



LangGraph was released on **January 17, 2024**, according to the human assistant's verification. For more details about the release or its features, you might explore the [GitHub releases page](https://github.com/langchain-ai/langgraph/releases). Let me know if you'd like further clarification!


In [4]:
snapshot = graph.get_state(config)

{k: v for k, v in snapshot.values.items() if k in ("name", "birthday")}

{'name': 'LangGraph', 'birthday': 'Jan 17, 2024'}

In [5]:
graph.update_state(config, {"name": "LangGraph (library)"})

snapshot = graph.get_state(config)

{k: v for k, v in snapshot.values.items() if k in ("name", "birthday")}

{'name': 'LangGraph (library)', 'birthday': 'Jan 17, 2024'}

测试

In [6]:
graph.invoke(
    {"messages": [{"role": "user", "content": "如何使用LangGraph来构建一个 AI 智能体？请给出具体的步骤？"}]},
    config,
)


{'messages': [HumanMessage(content='Can you look up when LangGraph was released? When you have the answer, use the human_assistance tool for review.', additional_kwargs={}, response_metadata={}, id='fb32e86d-4f74-4d1e-9764-a37ef4c1c4c0'),
  AIMessage(content='\n\n', additional_kwargs={'tool_calls': [{'id': '0198e61441983ae72eac646c895171ea', 'function': {'arguments': ' {"query": "LangGraph release date", "search_depth": "advanced", "time_range": "year"}', 'name': 'tavily_search'}, 'type': 'function', 'index': 0}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 1999, 'prompt_tokens': 1914, 'total_tokens': 3913, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 1974, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': None}, 'model_name': 'Qwen/Qwen3-8B', 'system_fingerprint': '', 'id': '0198e6136008a69b669afb5dab505e8f', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, i

In [7]:
user_input = "如何使用LangGraph来构建一个 AI 智能体？请给出具体的步骤？"
config = {"configurable": {"thread_id": "1"}}

events = graph.stream(
    {"messages": [{"role": "user", "content": user_input}]},
    config,
    stream_mode="values",
)
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

================================ Human Message =================================

如何使用LangGraph来构建一个 AI 智能体？请给出具体的步骤？
================================== Ai Message ==================================



以下是使用LangGraph构建AI智能体的具体步骤：

---

### **步骤 1: 环境准备**
1. **创建虚拟环境**  
   ```bash
   python -m venv langgraph_env
   source langgraph_env/bin/activate  # Linux/Mac  
   langgraph_env\Scripts\activate      # Windows  
   ```
2. **安装依赖**  
   ```bash
   pip install langgraph langchain streamlit
   ```

---

### **步骤 2: 定义AI智能体**
1. **创建代理逻辑**  
   在`src/agents`目录下，编写代理逻辑文件（如`research_agent.py`）：  
   ```python
   from langgraph.prebuilt import create_react_agent
   from langchain.chat import ChatOpenAI
   from langchain.tools import Tool

   llm = ChatOpenAI(model="gpt-3.5-turbo")
   tools = [Tool(name="search", func=search_function, description="搜索工具")]

   agent = create_react_agent(llm, tools, ...)
   ```
2. **注册代理到服务**  
   在`src/agents/agents.py`中，将代理添加到全局字典：  
   ```python
   agents = 